In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
file_path = "/content/drive/MyDrive/Team A Data/ERMICodeData.csv"
# file_path = "/content/drive/MyDrive/Team A Data/weinspect_cleaned.csv"

In [4]:
df = pd.read_csv(file_path)

In [5]:
df.head()

,City,State,Zip,Aspergillus flavus/oryzae,Aspergillus fumigatus,Aspergillus niger,Aspergillus ochraceus,Aspergillus penicillioides,Aspergillus restrictus*,Aspergillus sclerotiorum,...,"Eyes, Ears, Nose, & Throat",Resipiratory,Digestive,Circulatory,Skin,Brain,Nervous,Urinary,Immune,Reproductive
0,Mountain Brook,Alabama,35223,ND,ND,14,ND,NaN,11,Nd,...,"Sneezing/Allergies, Dry/Irritated eyes, Chroni...",NaN,"Diarrhea/Constipation, Bloating, Abdominal pai...","Raynaud's phenomenon, Low or reactive blood pr...","Itchy/Peeling skin, Skin rash, Fungal infections",Brain fog,"Anxiousness, Fatigue",Overactive bladder,NaN,"Changes in menstrual cycle, Inconsistent menst..."
1,Wasilla,Alaska,99623,ND,ND,12,ND,5,ND,ND,...,"Sneezing/Allergies, Runny nose/Post-nasal drip...","Shortness of breath, Chronic dry cough, Chroni...","Significant weight gain/loss, Diarrhea/Constip...","Cherry angiomas, Easy bruising/bleeding","Itchy/Peeling skin, Skin rash, Eczema","Brain fog, Memory loss, Anger/Aggressiveness","Anxiousness, Depression, Headache, Migraine, I...",NaN,Long-lasting colds,"Vaginal yeast or bacterial infections, Hormone..."
2,Soldotna,Alaska,99669-8654,ND,3,3,234,NaN,6,ND,...,"Sneezing/Allergies, Runny nose/Post-nasal drip...","Shortness of breath, Heaviness in the chest","Significant weight gain/loss, Bloating, Sweet ...","Spider veins, Easy bruising/bleeding, Iron def...","Itchy/Peeling skin, Flushing, Fungal infections","Brain fog, Confusion, Memory loss, Trouble fin...","Anxiousness, Depression, Headache, Insomnia, F...",Overactive bladder,"Increased susceptibility to infection, Chronic...","Changes in menstrual cycle, Inconsistent menst..."
3,Soldotna,Alaska,99669,ND,ND,ND,ND,NaN,ND,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Anchorage,Alaska,99508-4014,64,5,32,ND,NaN,6,ND,...,"Runny nose/Post-nasal drip, Ear popping/ringin...",NaN,Significant weight gain/loss,"Spider veins, Cherry angiomas, Raynaud's pheno...",Eczema,"Brain fog, Trouble finding the right word, Ang...","Anxiousness, Fatigue",NaN,NaN,NaN


In [6]:
def remove_asterisks(s):
  if not isinstance(s, float) and '*' in s:
    return s.replace("*", "")
  return s

def convert_to_int_or_zero(value):
  try:
    return int(value)
  except ValueError:
    return 0

for column_name in df.columns[3:40]:
  df[column_name] = df[column_name].apply(remove_asterisks)
  df[column_name] = df[column_name].apply(convert_to_int_or_zero)
  df[column_name] = df[column_name].fillna(0)

In [7]:
def is_column_uniform(column): # function to check if all values in a column are the same
    return column.nunique() == 1

# iterate through the columns of the DataFrame and drop those that are uniform
columns_to_drop = [col for col in df.columns if is_column_uniform(df[col])]
df.drop(columns=columns_to_drop, inplace=True)

In [8]:
import re
def remove_ie_from_Candidiasis(s):
  if isinstance(s, str):  # Check if the value is a string
      return re.sub(r'\([^)]*\)', '', s)
  else:
      return s

df['Diagnoses'] = df['Diagnoses'].apply(remove_ie_from_Candidiasis)

print(df['Diagnoses'])

0                    Hyperthyroid/Hypothyroid, IBS, PCOS
1                                                    NaN
2                                             CIRS, MCAS
3      Auto-Immune Disease, Lyme Disease, Hyperthyroi...
4      CIRS, Hyperthyroid/Hypothyroid, PCOS, Candidia...
                             ...                        
576                                                 MCAS
577                                         Lyme Disease
578                             Lyme Disease, Hashimotos
579    Auto-Immune Disease, Hashimotos, Chronic Fatig...
580        Auto-Immune Disease, Lyme Disease, Hashimotos
Name: Diagnoses, Length: 581, dtype: object


In [10]:
columns_to_encode = ['Eyes, Ears, Nose, & Throat', 'Resipiratory', 'Digestive',
       'Circulatory', 'Skin', 'Brain', 'Nervous', 'Urinary', 'Immune',
       'Reproductive']

# new df to store one-hot encoded results
encoded_df = pd.DataFrame()

# iterate through the columns and apply one-hot encoding
for column in columns_to_encode:
  encoded = df[column].str.get_dummies(', ')
  encoded_df = pd.concat([encoded_df, encoded], axis=1)

result_df = pd.concat([df, encoded_df], axis=1)
result_df.head()
df = result_df
columns_to_remove = ['Unnamed: 42', 'Unnamed: 43']
df = df.drop(columns='Diagnoses')
df = df.drop(columns=columns_to_encode)
df = df.drop(columns=columns_to_remove)

In [11]:
print(df.dtypes)

City                                     object
State                                    object
Zip                                      object
Aspergillus flavus/oryzae                 int64
Aspergillus fumigatus                     int64
                                          ...  
Hormone imbalances                        int64
Inconsistent menstrual cycle              int64
Infertility in both genders               int64
Jock itch                                 int64
Vaginal yeast or bacterial infections     int64
Length: 206, dtype: object


In [12]:
df = df.dropna()

In [13]:
df.head()

,City,State,Zip,Aspergillus flavus/oryzae,Aspergillus fumigatus,Aspergillus niger,Aspergillus ochraceus,Aspergillus penicillioides,Aspergillus restrictus*,Aspergillus sclerotiorum,...,Increased susceptibility to cancer,Increased susceptibility to infection,Long-lasting colds,Viral infections become bacterial,Changes in menstrual cycle,Hormone imbalances,Inconsistent menstrual cycle,Infertility in both genders,Jock itch,Vaginal yeast or bacterial infections
0,Mountain Brook,Alabama,35223,0,0,14,0,0,11,0,...,0,0,0,0,1,1,1,1,1,1
1,Wasilla,Alaska,99623,0,0,12,0,5,0,0,...,0,0,1,0,0,1,0,0,0,1
2,Soldotna,Alaska,99669-8654,0,3,3,234,0,6,0,...,0,1,0,0,1,1,1,0,0,0
3,Soldotna,Alaska,99669,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Anchorage,Alaska,99508-4014,64,5,32,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# once done with data manipulation in df, push changes to csv file using
df.to_csv('/content/drive/My Drive/Colab Notebooks/Tiffany Dataset.csv', index=False)